# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,0.52,41,87,0.89,US,1738787567
1,1,fortaleza,-3.7227,-38.5247,28.07,78,75,4.63,BR,1738787568
2,2,adamstown,-25.0660,-130.1015,25.23,78,4,6.10,PN,1738787569
3,3,usogorsk,63.4106,48.6872,-13.39,100,94,1.94,RU,1738787570
4,4,grytviken,-54.2811,-36.5092,1.80,94,99,7.01,GS,1738787571


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity", 
    color = "City"
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 22) & (city_data_df["Max Temp"] < 30) &  
    (city_data_df["Wind Speed"] < 5) &  
    (city_data_df["Cloudiness"] <= 1)  
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,lazaro cardenas,17.9583,-102.2000,28.82,53,0,4.20,MX,1738787624
182,182,minas de matahambre,22.5887,-83.9447,27.98,47,1,4.67,CU,1738787778
224,224,rio verde,21.9333,-99.9833,27.85,37,0,2.96,MX,1738787827
229,229,crane,31.3974,-102.3501,28.47,18,0,3.62,US,1738787833
279,279,saint-pierre,-21.3393,55.4781,27.82,78,0,2.06,RE,1738787894


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = {}

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
48,lazaro cardenas,MX,17.9583,-102.2000,53,NaN
182,minas de matahambre,CU,22.5887,-83.9447,47,NaN
224,rio verde,MX,21.9333,-99.9833,37,NaN
229,crane,US,31.3974,-102.3501,18,NaN
279,saint-pierre,RE,-21.3393,55.4781,78,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"

    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    #print(f"{base_url}?{requests.compat.urlencode(params)}")
    #print(f"Status Code: {response.status_code}")

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
minas de matahambre - nearest hotel: No hotel found
rio verde - nearest hotel: Hotel Vista Bonita
crane - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
san luis de la loma - nearest hotel: No hotel found
sinfra - nearest hotel: Complexe wilson
cabo san lucas - nearest hotel: Comfort Rooms
broken hill - nearest hotel: Ibis Styles
sansanne-mango - nearest hotel: Le Matoush
viedma - nearest hotel: Níjar


,City,Country,Lat,Lng,Humidity,Hotel Name
48,lazaro cardenas,MX,17.9583,-102.2000,53,Hotel Sol del Pacífico
182,minas de matahambre,CU,22.5887,-83.9447,47,No hotel found
224,rio verde,MX,21.9333,-99.9833,37,Hotel Vista Bonita
229,crane,US,31.3974,-102.3501,18,No hotel found
279,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
301,san luis de la loma,MX,17.2706,-100.8939,51,No hotel found
313,sinfra,CI,6.6167,-5.9167,23,Complexe wilson
335,cabo san lucas,MX,22.8909,-109.9124,38,Comfort Rooms
341,broken hill,AU,-31.9500,141.4333,52,Ibis Styles
360,sansanne-mango,TG,10.3592,0.4708,9,Le Matoush


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity", 
    color ="City",
    hover_cols=["City", "Country", "Hotel Name", "Humidity"] 
)
# Display the map
map_plot2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)